In [1]:
#!pip install pandas_market_calendars
import pandas as pd
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import os
import math



datasetPath = "C:\\Users\\bendi\\Desktop\\dataset"

tradingDaysFolders = os.listdir(os.path.join(datasetPath, "aggregatePerDay"))

csvFilePaths = [os.path.join(datasetPath, "aggregatePerDay", folderName, folderName+".csv") 
                for folderName in tradingDaysFolders]

csvFilePaths.reverse()

In [2]:
calendar = mcal.get_calendar("CBOE_Index_Options")
tradingDays = calendar.schedule(
    datetime(2013, 4, 10), 
    datetime(2030, 1, 1)).index

def countTradingDays(start_date, end_date):
    filtered_days = tradingDays[(tradingDays >= start_date) & (tradingDays <= end_date)]
    return len(filtered_days) - 1

# Calculate nDTE% 
For each day: "0dte_cnt", "1dte_cnt", "2dte_cnt", "3dte_cnt", "4dte_cnt", "5to21dte_cnt", "22to43dte_cnt", "44to64dte_cnt", "higher_dte_cnt"

In [3]:
print(datetime.now())

# One row per trading day: Will store the number of Ndte options traded and their percentage of the total options trading volume
dailyNdteOptionsVolume = pd.DataFrame(columns=["0dte_put_cnt", 
                                               "1dte_put_cnt", 
                                               "2dte_put_cnt", 
                                               "3dte_put_cnt", 
                                               "4dte_put_cnt", 
                                               "5to21dte_put_cnt", 
                                               "22to43dte_put_cnt", 
                                               "44to64dte_put_cnt", 
                                               "higher_dte_put_cnt", 
                                               
                                               "0dte_call_cnt", 
                                               "1dte_call_cnt", 
                                               "2dte_call_cnt", 
                                               "3dte_call_cnt", 
                                               "4dte_call_cnt", 
                                               "5to21dte_call_cnt", 
                                               "22to43dte_call_cnt", 
                                               "44to64dte_call_cnt", 
                                               "higher_dte_call_cnt"])

# for each trading day in the dataset (one csv file per day)
for n, path in enumerate(csvFilePaths):
    df = pd.read_csv(path)
    
    #count Ndte trades
    day = {"0dte_put_cnt": 0,
           "1dte_put_cnt": 0,
           "2dte_put_cnt": 0,
           "3dte_put_cnt": 0,
           "4dte_put_cnt": 0,
           "5to21dte_put_cnt": 0,
           "22to43dte_put_cnt": 0,
           "44to64dte_put_cnt": 0,
           "higher_dte_put_cnt": 0,
           
           "0dte_call_cnt": 0,
           "1dte_call_cnt": 0,
           "2dte_call_cnt": 0,
           "3dte_call_cnt": 0,
           "4dte_call_cnt": 0,
           "5to21dte_call_cnt": 0,
           "22to43dte_call_cnt": 0,
           "44to64dte_call_cnt": 0,
           "higher_dte_call_cnt": 0}
    
    for index, row in df.iterrows():
        volume = row['trade_volume']

        quote_datestr = row['quote_date']
        expirationstr = row['expiration']

        quote_date = datetime.strptime(quote_datestr, "%Y-%m-%d")
        expiration = datetime.strptime(expirationstr, "%Y-%m-%d")

        ndte = countTradingDays(quote_date, expiration)
        
        option_type = "put" if row['option_type']=="P" else "call"

        if ndte <= 4:    # equivalent of 0-7 including weekends
            day[f"{ndte}dte_{option_type}_cnt"] += volume
        elif ndte <= 21: # equivalent of 30 including weekends
            day[f"5to21dte_{option_type}_cnt"] += volume
        elif ndte <= 43: # equivalent of 60 including weekends
            day[f"22to43dte_{option_type}_cnt"] += volume
        elif ndte <= 64: # equivalent of 90 including weekends
            day[f"44to64dte_{option_type}_cnt"] += volume
        else:
            day[f"higher_dte_{option_type}_cnt"] += volume
    
    # add this trading day's data to the master dataframe
    row = pd.DataFrame([day])
    row.index = [quote_date]
    
    dailyNdteOptionsVolume = pd.concat([dailyNdteOptionsVolume, row])
    
    #save to file every 100
    if n%100 == 0:
        print(n)
        dailyNdteOptionsVolume = dailyNdteOptionsVolume.rename_axis("date")
        dailyNdteOptionsVolume.to_csv("backup.csv")
               
print(datetime.now())

2023-10-23 22:15:40.179081


C:\Users\bendi\AppData\Local\Temp\ipykernel_15496\4174443859.py:77: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  dailyNdteOptionsVolume = pd.concat([dailyNdteOptionsVolume, row])


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2023-10-24 00:17:23.073767


Zero trades: 2018-03-12 <br>
Zero trades: 2017-11-24 <br>
Zero trades: 2017-11-21 <br><br>

# Write to csv

In [4]:
# Giving a name to the index
dailyNdteOptionsVolume = dailyNdteOptionsVolume.rename_axis("date")
# Save the DataFrame to a CSV file
dailyNdteOptionsVolume.to_csv("output.csv")

# SPX Volatility

In [12]:
# exclude the closing price of the previous day in the calculation of tha daily volatility?
intraday = True

# Read CSV into a DataFrame
dailyNdteOptionsVolume = pd.read_csv("dailyNdteOptionsVolume.csv", parse_dates=['date'], dayfirst=True, index_col='date')

# Read Historic price data into a DataFrame
path = os.path.join(datasetPath, "spx.xlsx")
spx30minPrice = pd.read_excel(path, sheet_name="spx30min", parse_dates=['date'], index_col='date')
# reverse the dataframe, such that it goes from 2013 to 2023 (important for calculation)
spx30minPrice = spx30minPrice.iloc[::-1]



# calculate volatility for each day in our options dataset
calendar = mcal.get_calendar("CBOE_Index_Options")
optionsTradingDays = calendar.schedule(
    datetime(2013,4,10), # max and min from dataset
    datetime(2023,10,3)).index

spxIntradayVolatility = pd.DataFrame(columns=["volatility", "return"])

for n,day in enumerate(optionsTradingDays):    
    # Extract data for the current day. Does not inlcude prev day's closing price. We want to stuy the impact on intraday vol.
    day_data = spx30minPrice[spx30minPrice.index.date == day.date()]
    
    # add prev day close price
    if n!=0 and not intraday:
        day_data = pd.concat([prevDayClose, day_data])

    # Calculate intraday volatility
    intraday_volatility = math.sqrt( day_data['last_price'].pct_change().std() )
    
    # Calculate intraday return
    if not day_data.empty:
        opening_price = day_data.iloc[0]['last_price']
        closing_price = day_data.iloc[-1]['last_price']
        intraday_return = (closing_price - opening_price) / opening_price
    else:
        intraday_return = 0
    
    # add this trading day's data to the master dataframe
    row = pd.DataFrame([{"volatility": intraday_volatility, "return": intraday_return}])
    row.index = [day.date()]
    
    spxIntradayVolatility = pd.concat([spxIntradayVolatility, row])
    
    # save close
    if not intraday: # dette blir ikke riktig hvis annen tidssjone use .iloc[0]['last_price']
        prevDayClose = day_data[(day_data.index.hour == 22)].tail(1)
    
# Giving a name to the index and reverse again so that the dataframe goes from 2023 to 2013
spxIntradayVolatility = spxIntradayVolatility.rename_axis("date")[::-1]

# Write to CSV

In [13]:
# Save the DataFrame to a CSV file
spxIntradayVolatility.to_csv(f"spx{'Intraday' if intraday else ''}Volatility.csv")

# Intraday 0DTE%
Calculate nDTE_cnt for every 1 min aggregate volume 

Separate put and call options

In [15]:
path = os.path.join(datasetPath, "UnderlyingOptionsTradesCalcs_2023-08-15", "UnderlyingOptionsTradesCalcs_2023-08-15.csv")
data = pd.read_csv(path, parse_dates=['quote_datetime', 'expiration'])

In [16]:
# sort trades after quote_datetime
data = data.sort_values(by='quote_datetime')
# filter out pre trading
data = data[data["quote_datetime"] >= datetime(2023, 8, 15, 9, 30)]

In [17]:
data

,underlying_symbol,quote_datetime,sequence_number,root,expiration,strike,option_type,exchange_id,trade_size,trade_price,...,trade_iv,trade_delta,underlying_bid,underlying_ask,number_of_exchanges,{exchange,bid_size,bid,ask_size,ask}[number_of_exchanges]
7866,^SPX,2023-08-15 09:30:02.156,1389257,SPX,2023-09-15,2400.0,P,5,1,0.15,...,0.6883,-0.0006,4473.1475,4473.1475,0,NaN,NaN,NaN,NaN,NaN
22914,^SPX,2023-08-15 09:30:02.305,1438065,SPX,2024-01-19,3500.0,P,5,1,22.20,...,0.2755,-0.0609,4473.1475,4473.1475,0,NaN,NaN,NaN,NaN,NaN
531771,^SPX,2023-08-15 09:30:02.343,1446980,SPXW,2023-08-18,4480.0,P,5,1,28.90,...,0.1509,-0.5374,4473.1475,4473.1475,0,NaN,NaN,NaN,NaN,NaN
531525,^SPX,2023-08-15 09:30:02.343,1446920,SPXW,2023-08-18,4480.0,C,5,1,21.90,...,0.1480,0.4618,4473.1475,4473.1475,0,NaN,NaN,NaN,NaN,NaN
17530,^SPX,2023-08-15 09:30:02.392,1458174,SPX,2023-10-20,4200.0,P,5,1,34.74,...,0.1864,-0.1771,4473.1475,4473.1475,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457323,^SPX,2023-08-15 16:59:39.805,2981751029,SPXW,2023-08-16,4405.0,P,5,50,4.23,...,0.1687,-0.1990,4437.2651,4437.2651,0,NaN,NaN,NaN,NaN,NaN
585326,^SPX,2023-08-15 16:59:44.320,2981757785,SPXW,2023-10-20,4100.0,C,5,1,399.89,...,0.2045,0.8515,4437.2651,4437.2651,0,NaN,NaN,NaN,NaN,NaN
585712,^SPX,2023-08-15 16:59:44.320,2981757786,SPXW,2023-10-20,4475.0,C,5,1,105.79,...,0.1421,0.5098,4437.2651,4437.2651,0,NaN,NaN,NaN,NaN,NaN
530209,^SPX,2023-08-15 16:59:48.382,2981763879,SPXW,2023-08-18,4465.0,C,5,1,13.60,...,0.1526,0.3397,4437.2651,4437.2651,0,NaN,NaN,NaN,NaN,NaN


In [18]:
nDTEvolume = pd.DataFrame(columns=['quote_minute', 'ndte', 'option_type', 'volume'])
i = 0

for index, row in data.iterrows():
    
    quote_minute = datetime(
        row.quote_datetime.year, 
        row.quote_datetime.month, 
        row.quote_datetime.day, 
        row.quote_datetime.hour, 
        row.quote_datetime.minute)
    
    option_type = row.option_type
    
    quote_date = datetime(2023, 8, 15)
    expiration = row.expiration
    ndte = countTradingDays(quote_date, expiration)
    
    volume = row.trade_size
    
    # add data to df
    
    mask = (
        (nDTEvolume['quote_minute'] == quote_minute) &
        (nDTEvolume['ndte']         == ndte        ) &
        (nDTEvolume['option_type']  == option_type )
    )

    # If the combination exists, update the 'volume'
    if nDTEvolume[mask].shape[0] > 0:
        nDTEvolume.loc[mask, 'volume'] += volume
    else:
        # If the combination doesn't exist, add a new row
        nDTEvolume.loc[i] = [quote_minute, ndte, option_type, volume]
        i += 1

# Write to CSV

In [19]:
# Save the DataFrame to a CSV file
nDTEvolume = nDTEvolume.rename_axis("index")
nDTEvolume.to_csv(f"nDTEvolume.csv")

# Calculate Option Delta
delta option price / delta index price

In [48]:
path = os.path.join(datasetPath, "spx.xlsx")
spx30minPrice = pd.read_excel(path, sheet_name="spx30min", parse_dates=['date'], index_col='date')

def getEndOfDayPrice(date):
    
    return spx30minPrice[spx30minPrice.index.date == date.date()].iloc[0]['last_price']
    #timestamp = date+timedelta(hours=22) #return spx30minPrice.loc[timestamp, 'last_price']

In [49]:
ndteLst = [0,1,2,3,4]
# I only want options that are between start and end % in the money(strike_index_ratio_x)
start_strike_index_ratio_x_lst = [0, 0.01, 0.05, 0.1, 0.2]


# Create an empty DataFrame with columns
results = pd.DataFrame(columns=['date', 'ndte', 'strike_index_ratio', 'mean_delta'])
index = 0

for n in range(len(csvFilePaths) - 1):
    path1 = csvFilePaths[n]
    path2 = csvFilePaths[n+1]
    df1 = pd.read_csv(path1, parse_dates=['quote_date', 'expiration'])
    df2 = pd.read_csv(path2, parse_dates=['quote_date', 'expiration'])

    # get index price change
    date1 = df1['quote_date'].iloc[0]
    date2 = df2['quote_date'].iloc[0]

    index_price1 = getEndOfDayPrice(date1)
    index_price2 = getEndOfDayPrice(date2)

    # calculate in the money %
    df1['strike_index_ratio'] = df1['strike'] / index_price1
    df2['strike_index_ratio'] = df2['strike'] / index_price2

    # calculate ndte
    df1['ndte'] = df1.apply(lambda row: countTradingDays(row['quote_date'], row['expiration']), axis=1)
    df2['ndte'] = df2.apply(lambda row: countTradingDays(row['quote_date'], row['expiration']), axis=1)

    # Filter out rows where "bid_eod" or "ask_eod" is equal to 0
    df1 = df1[(df1['bid_eod'] != 0) & (df1['ask_eod'] != 0)]
    df2 = df2[(df2['bid_eod'] != 0) & (df2['ask_eod'] != 0)]

    # Filter out out of the money options. does not make sense to take averge of deltas of both in and out of the money options
    def is_out_of_money(row):
        if row['option_type'] == 'C':
            return row['strike'] > index_price1
        elif row['option_type'] == 'P':
            return row['strike'] < index_price1
        else:
            return False  # Handle other cases if needed

    # filter out out of the money
    mask1 = df1.apply(is_out_of_money, axis=1)
    mask2 = df2.apply(is_out_of_money, axis=1)
    df1 = df1[~mask1]
    df2 = df2[~mask2]

    # calculate mid price
    df1['mid_price_eod'] = df1[['bid_eod', 'ask_eod']].mean(axis=1)
    df2['mid_price_eod'] = df2[['bid_eod', 'ask_eod']].mean(axis=1)

    # group similar options with different "root" values. i.e. SPX and SPXW
    df1 = df1.groupby(['strike', 'expiration', 'option_type', 'ndte']).agg({'mid_price_eod': 'mean', 'quote_date': 'first', 'strike_index_ratio': 'first'}).reset_index()
    df2 = df2.groupby(['strike', 'expiration', 'option_type', 'ndte']).agg({'mid_price_eod': 'mean', 'quote_date': 'first', 'strike_index_ratio': 'first'}).reset_index()

    # inner join
    merged = pd.merge(df1, df2, on=['strike', 'expiration', 'option_type'], how='inner')
    merged["option_price_change"] = merged["mid_price_eod_x"] - merged["mid_price_eod_y"]

    index_price_change = index_price1 - index_price2

    # Filter out rows where index or option price change is equal to 0
    merged["index_price_change"] = index_price_change
    merged = merged[(merged['option_price_change'] != 0) & (merged['index_price_change'] != 0)]

    merged["option_delta"] = merged["option_price_change"] / index_price_change
    
    
    for i, ndte in enumerate(ndteLst):
        for j, start in enumerate(start_strike_index_ratio_x_lst):
            
            end = start + 0.005
    
            interval1_start = 1 - end
            interval1_end   = 1 - start
            interval2_start = 1 + start
            interval2_end   = 1 + end

            # Filter the DataFrame on similar strike_divided_by_index_price_x
            filtered = merged[((merged['strike_index_ratio_x'] >= interval1_start) & (merged['strike_index_ratio_x'] <= interval1_end)) | ((merged['strike_index_ratio_x'] >= interval2_start) & (merged['strike_index_ratio_x'] <= interval2_end))]

            # Make the 'Values' column positive
            filtered = filtered.copy()
            filtered['option_delta'] = filtered['option_delta'].abs()

            # # Define a threshold based on percentiles (e.g., removing top and bottom 5%)
            # lower_threshold = a['option_delta'].quantile(0.05)
            # upper_threshold = a['option_delta'].quantile(0.95)
            # 
            # # Filter out values outside the threshold
            # a = a[(a['option_delta'] >= lower_threshold) & (a['option_delta'] <= upper_threshold)]

            filtered = filtered[filtered['ndte_x']==2]
            mean = filtered['option_delta'].mean()

            results.loc[index] = [date1, ndte, start, mean]

            index += 1
            
    if n%100 == 0:
        print(n)
        results.to_csv("option_delta_backup.csv")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


# Write to CSV

In [53]:
# Giving a name to the index
results = results.rename_axis("index")
# Save the DataFrame to a CSV file
results.to_csv(f"option_delta.csv")

In [54]:
results

,date,ndte,strike_index_ratio,mean_delta
i,,,,
0,2023-10-03,0,0.00,0.512574
1,2023-10-03,0,0.01,0.722663
2,2023-10-03,0,0.05,0.981507
3,2023-10-03,0,0.10,0.980658
4,2023-10-03,0,0.20,NaN
...,...,...,...,...
65970,2013-04-11,4,0.00,NaN
65971,2013-04-11,4,0.01,NaN
65972,2013-04-11,4,0.05,NaN
